In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numexpr as en
sns.set()

ModuleNotFoundError: No module named 'torch'

In [2]:
from itertools import product
def combinationL(loop_val): 
    return np.array(list({i for i in product(*loop_val)}))
combinationL(([1,1,2,1],[1,1,2,1]))

array([[1, 1],
       [1, 2],
       [2, 1],
       [2, 2]])

## 创建数据

In [1]:
X = np.array([1,0,1,1,0])
Y = np.array([1,1,1,0,0])
A,B = np.array([0,0]),np.array([1,1]) 

NameError: name 'np' is not defined

## numpy list 转torch

In [4]:
def checkType(X):
    if isinstance(X,(np.ndarray,list,tuple)):
        return torch.FloatTensor(X)
    elif isinstance(X,(torch.TensorType,torch.FloatType)):
        return X
    else:
        print("Type Error")

# numpy 评估表达式求概率

In [5]:
def NumpyProb(X,symbol,x):
    n = X.size
    Lambda = "{}{}{}".format("X",symbol,"x")
    expr = en.evaluate(Lambda)
    return (expr).dot(np.ones(n))/n

# numpy 求解信息熵

In [6]:
def NumpyEntropy(X):
    NE = 0
    for x in np.unique(X):
        PX = NumpyProb(X,'==',x)
        NE += (- PX * np.log2(PX))
    return NE
NumpyEntropy(A)

0.0

# torch求解信息熵

In [7]:
def NumpyJointProb(X,Y):
    init_XY = np.c_[X,Y]
    distionXY = combinationL(init_XY.T)
    m,n = init_XY.shape
    dm,dn = distionXY.shape
    if dm ==1:
        distionXY = np.repeat(distionXY,dn,axis=0).T
        return np.array([((init_XY.T==xy).dot(np.ones(n))==n).dot(np.ones(m))/m for xy in distionXY])
    elif dm > 1:
        return np.array([((init_XY==xy).dot(np.ones(n))==n).dot(np.ones(m))/m for xy in distionXY])

NumpyJointProb(A,B),NumpyJointProb(X,Y)

(array([0.5, 0.5]), array([0.2, 0.4, 0.2, 0.2]))

# numpy求联合概率

In [8]:
def NumpyJointProb(X,Y):
    init_XY = np.c_[X,Y]
    distionXY = combinationL(init_XY.T)
    m,n = init_XY.shape
    dm,dn = distionXY.shape
    if dm ==1:
        distionXY = np.repeat(distionXY,dn,axis=0).T
        return np.array([((init_XY.T==xy).dot(np.ones(n))==n).dot(np.ones(m))/m for xy in distionXY])
    elif dm > 1:
        return np.array([((init_XY==xy).dot(np.ones(n))==n).dot(np.ones(m))/m for xy in distionXY])

NumpyJointProb(A,B),NumpyJointProb(X,Y)

(array([0.5, 0.5]), array([0.2, 0.4, 0.2, 0.2]))

# numpy求解联合熵
# $$ H(X,Y) = - \sum_{x \in X} \sum_{y \in Y} P(x,y)logP(x,y)  $$

In [9]:
def NumpyJointEntropy(X,Y):
    PXY = NumpyJointProb(X,Y)
    return (-PXY*np.log2(PXY)).sum()
    
NumpyJointEntropy(X,Y),NumpyJointEntropy(A,B)

(1.9219280948873623, 1.0)

In [10]:
NumpyJointEntropy(A,B),NumpyEntropy(A),NumpyEntropy(B)

(1.0, 0.0, 0.0)

# torch 求解联合概率

In [11]:
def TorchJointProb(X,Y):
    init_XY = np.c_[X,Y]
    distionXY = combinationL(init_XY.T)
    m,n = init_XY.shape
    dm,dn = distionXY.shape

    init_XY , distionXY = checkType(init_XY),checkType(distionXY)

    if dm ==1:
        distionXY = torch.repeat_interleave(distionXY,repeats=dn,dim=-1).reshape(n,-1)
        return torch.tensor([((init_XY.T==xy).float().matmul(torch.ones(n))==2).float().sum()/m for xy in distionXY])
    elif dm > 1:
        return torch.tensor([(((init_XY==xy).float().mm(torch.ones(n)[:,None]))==n).sum().float()/m for xy in distionXY])

TorchJointProb(A,B),TorchJointProb(X,Y),

(tensor([0.5000, 0.5000]), tensor([0.2000, 0.4000, 0.2000, 0.2000]))

# torch 求解联合熵

In [12]:
def TorchJointEntropy(X,Y):
    PXY = TorchJointProb(X,Y)
    return (-PXY*torch.log2(PXY)).sum()
TorchJointEntropy(X,Y),TorchJointEntropy(A,B)

(tensor(1.9219), tensor(1.))

# 求解互信息

In [13]:
def MutualInfo(X,Y,keyelement="X<->Y",keyType='numpy'):
    if keyType == 'numpy':
        HX,HY,HXY = NumpyEntropy(X),NumpyEntropy(Y),NumpyJointEntropy(X,Y)
    elif keyType == "torch":
        HX,HY,HXY =  TorchEntropy(X), TorchEntropy(Y),TorchJointEntropy(X,Y)    
    return {"X<-Y":HX-HXY,"X->Y":HY-HXY,"X<->Y":HX+HY-HXY}[keyelement]

## $$ I(X;Y) = H(X)+H(Y)-H(X,Y) $$
## 可以通过韦恩图换算得到

In [14]:
MutualInfo(X,Y,keyelement='X<->Y',keyType='torch')

NameError: name 'TorchEntropy' is not defined


## $$ I(X;Y) = H(Y)-H(X,Y) $$
## 表示知道随机变量X 随机变量Y熵的减少

In [15]:
MutualInfo(X,Y,keyelement='X->Y',keyType='torch')

NameError: name 'TorchEntropy' is not defined

## $$ I(X;Y) = H(X)-H(X,Y) $$
## 表示知道随机变量Y 随机变量X熵的减少 

In [16]:
MutualInfo(X,Y,keyelement='X<-Y',keyType='torch')

NameError: name 'TorchEntropy' is not defined

https://www.bilibili.com/video/BV194411X7Jg?p=1&share_medium=ipad&share_plat=ios&share_source=QQ&share_tag=s_i&timestamp=1593860517&unique_k=gWqiA3

# 2.0表达式求解互信息

# $$ I(X:Y) = \sum{ P(x,y)log\frac{p(x,y)}{p(x)p(y)} } $$ 

# 求联合概率

In [17]:
def MarginalProb(X,Y):
    init_JP = NumpyJointProb(X,Y).reshape(2,-1).T
    dm,dn = init_JP.shape
    if dm == 1:
        return init_JP[0]
    else:
        return sum(init_JP)
MarginalProb(X,Y),MarginalProb(A,B)

(array([0.6, 0.4]), array([0.5, 0.5]))

# 求互信息

In [86]:
def MutualInfoExpr(X,Y):
    PXY = NumpyJointProb(X,Y)
    PX,PY = tuple(MarginalProb(X,Y))
    return PXY,PX,PY
MutualInfoExpr(X,Y)

(array([0.2, 0.2, 0.2, 0.4]), 0.4, 0.6000000000000001)

In [87]:
MutualInfoExpr(A,B),MutualInfo(A,B,keyelement="X<->Y",keyType='torch')

((array([0.5, 0.5]), 0.5, 0.5), tensor(-1.))

In [88]:
NumpyJointProb(A,B)

array([0.5, 0.5])

In [89]:
NumpyProb(X,'==',0),NumpyProb(X,'==',1),NumpyProb(Y,'==',0),NumpyProb(Y,'==',1)

(0.4, 0.6, 0.4, 0.6)

In [90]:
((0.2+0.4)*(0.2+0.4))*np.log2((0.2+0.4)*(0.2+0.4)/(0.4*0.6))

0.21058650025961645